In [511]:
# Import 
import pandas as pd
import numpy as np

In [512]:
column_names = ['common_name', 'season', 'matches_played', 'wins', 'draws', 'losses', 'league_position', 'goals_scored', 'goals_conceded', 'goal_difference']
countries = ['england', 'france', 'germany', 'italy', 'spain']
league = {
        'england': 'premier-league',
        'france': 'ligue-1',
        'germany': 'bundesliga',
        'italy': 'serie-a',
        'spain': 'la-liga'
    }
seasons = [year for year in range(2010, 2020)]
match_column_names = ['home_team_name', 'away_team_name', 'home_team_goal_count', 'away_team_goal_count', 'season']

In [513]:
def clean_league_data():
    countries_column_names = ['cID', 'country_name', 'number_of_teams']
    teams_column_names = ['tID', 'cID', 'common_name']
    all_teams = pd.DataFrame(columns = teams_column_names)
    all_countries = pd.DataFrame(columns = countries_column_names)
    for c_id in range(len(countries)):
        country = countries[c_id]
        full_df = pd.DataFrame(columns = column_names)
        for season in seasons:
            address = '{folder}/{country}-{league}-teams-{start}-to-{end}-stats.csv'.format(folder = country, country = country, league = league[country], start = season, end = season + 1)
            try:
                temp_df = pd.read_csv(address)[column_names]
                full_df = pd.concat([full_df, temp_df])
            except FileNotFoundError:
                print('{league} in {season} is not found'.format(league = league[country], season = season))
        full_df['season'] = full_df['season'].str[:4]
        full_df.to_csv('cleaned_data/{league}.csv'.format(league = league[country]), index=False)
        number_of_teams = full_df['common_name'].nunique() 
        teams = full_df['common_name'].unique() 
        country_dict = {
            'cID' : [c_id],
            'country_name' : [country],
            'number_of_teams': [number_of_teams]
        }
        all_countries = all_countries.append(pd.DataFrame(country_dict))
        team_dict = {
            'tID': [0 for i in range(number_of_teams)],
            'common_name':  list(teams),
            'cID': [c_id for i in range(number_of_teams)]
        }
        all_teams = all_teams.append(pd.DataFrame(team_dict))
    all_countries.to_csv('cleaned_data/countries.csv', index=False)
    all_teams.to_csv('cleaned_data/teams.csv', index=False)

In [514]:
clean_league_data()

serie-a in 2010 is not found


In [515]:
def clean_champion_league_data():
    if 'league_position' in column_names:
        column_names.remove('league_position')
    for competition in ['europa-league', 'champions-league']:
        full_df_team = pd.DataFrame(columns = column_names)
        full_df_match = pd.DataFrame(columns = match_column_names)
        for season in seasons:
            address_team = '{folder}_team/europe-uefa-{competition}-teams-{start}-to-{end}-stats.csv'.format(folder = competition, competition = competition, start = season, end = season + 1)
            try:
                temp_df_team = pd.read_csv(address_team)
                temp_df_team['season'] = season
                temp_df_team = temp_df_team.loc[temp_df_team['country'].str.lower().isin(countries)][column_names]
                full_df_team = pd.concat([full_df_team, temp_df_team])
            except FileNotFoundError:
                 print('{competition} in {season} is not found (teams)'.format(competition = competition, season = season))

            address_match = '{folder}_match/europe-uefa-{competition}-matches-{start}-to-{end}-stats.csv'.format(folder = competition, competition = competition, start = season, end = season + 1)
            try:
                temp_df_match = pd.read_csv(address_match)
                temp_df_match['season'] = season
                temp_df_match = temp_df_match[match_column_names]
                full_df_match = pd.concat([full_df_match, temp_df_match])
            except FileNotFoundError:
                print('{competition} in {season} is not found (competition)'.\
                format(competition = competition,season = season)) 
        full_df_team.to_csv('cleaned_data/{competition}_team.csv'.\
        format(competition = competition), index = False)
        full_df_match.to_csv('cleaned_data/{competition}_match.csv'.\
        format(competition = competition), index = False)

In [516]:
clean_champion_league_data()

europa-league in 2015 is not found (teams)
europa-league in 2015 is not found (competition)
champions-league in 2019 is not found (competition)


In [517]:
data = pd.read_csv('cleaned_data/champions-league_team.csv')

In [518]:
# Create a database of all teams
def team_data():
    teams = []
    leagues = list(league.values())
    country = []
    team_country_table = {}
    for l in range(5):
        teams_in_league = set(pd.read_csv('cleaned_data/{le}.csv'.format(le = leagues[l]))['common_name'])
        teams = teams + list(teams_in_league)
        team_country_table[leagues[l]] = teams_in_league
        country.extend([l for i in range(len(teams_in_league))])
    team_data = pd.DataFrame(data = {'name': list(teams), 'cID': country})
    team_data= team_data.sort_values(by=['name'])
    team_data['tID'] = np.arange(1, len(team_data) + 1)
    team_data = team_data[['tID', 'cID', 'name']]
    team_data.to_csv('cleaned_data/team.csv', index = False)

In [519]:
team_data()

In [520]:
# Apply index to all datasets
# Champions league 
def modify_name():
    team_data = pd.read_csv('cleaned_data/team.csv')
    for competition in ['europa-league_team', 'champions-league_team', 'premier-league', 'ligue-1', 'bundesliga',\
        'serie-a', 'la-liga']:
        competition_data = pd.read_csv('cleaned_data/{c}.csv'.format(c = competition))
        competition_data = team_data.merge(competition_data, \
            left_on='name', right_on='common_name', left_index=False, right_index = False)
        competition_data = competition_data.drop(columns = ['common_name', 'name'])   
        competition_data.to_csv('cleaned_data/{c}.csv'.format(c = competition), index = False)
    for match in ['champions-league_match', 'europa-league_match']:
        match_data = pd.read_csv('cleaned_data/{c}.csv'.format(c = match))
        match_data = match_data.merge(team_data, left_on='home_team_name', right_on='name').\
            drop(['home_team_name', 'name'], axis = 1).rename(columns = {'tID': 'home_team', 'cID': 'home_country'})
        match_data = match_data.merge(team_data, left_on='away_team_name', right_on='name').\
            drop(['away_team_name', 'name'],axis = 1).rename(columns = {'tID': 'away_team', 'cID': 'away_country'})
        match_data.to_csv('cleaned_data/{c}.csv'.format(c = match), index = False)

In [521]:
modify_name()

In [522]:
champion_league_team  = pd.read_csv('cleaned_data/champions-league_team.csv')
europa_league_team = pd.read_csv('cleaned_data/europa-league_team.csv')

In [523]:
# dic = {'premier-league' : 0, 'ligue-1': 1, 'bundesliga': 2, 'serie-a': 3, 'la-liga': 4}
champion_league_team.groupby(['cID']).apply(lambda x : sum(x['wins'] / sum(x['matches_played'])))

cID
0    0.507979
1    0.388601
2    0.505988
3    0.418327
4    0.553922
dtype: float64

In [524]:
europa_league_team.groupby(['cID']).apply(lambda x : sum(x['wins'] / sum(x['matches_played'])))

cID
0    0.546075
1    0.405063
2    0.472325
3    0.467290
4    0.575221
dtype: float64

In [525]:
europa_league_match = pd.read_csv('cleaned_data/champions-league_match.csv')

In [526]:
def grouped():
    for competition in ['europa-league_match', 'champions-league_match']:
        match_df = pd.read_csv('cleaned_data/{c}.csv'.format(c = competition))
        match_df['win_team'] = europa_league_match.apply(lambda x: 'home' \
            if x['home_team_goal_count'] > x['away_team_goal_count'] else ('draw' if x['home_team_goal_count']\
                == x['away_team_goal_count'] else 'away'), axis = 1)
        matchdf_grouped = match_df.groupby(['home_country', 'away_country'])
        print(matchdf_grouped.apply(lambda x : (len(x), sum(x['win_team'] == 'home'), sum(x['win_team'] == 'away'),\
             sum(x['win_team'] == 'draw'))))

In [527]:
# dic = {'premier-league' : 0, 'ligue-1': 1, 'bundesliga': 2, 'serie-a': 3, 'la-liga': 4}
grouped()

home_country  away_country
0             0                (1, 1, 0, 0)
              1                (6, 4, 1, 1)
              2                (4, 2, 0, 2)
              3               (12, 9, 0, 3)
              4                (8, 6, 2, 0)
1             0                (6, 4, 1, 1)
              2               (10, 5, 4, 1)
              3                (9, 3, 4, 2)
              4                (9, 2, 4, 3)
2             0                (4, 2, 2, 0)
              1               (10, 4, 3, 3)
              2                (2, 0, 2, 0)
              3               (10, 5, 3, 2)
              4               (12, 6, 6, 0)
3             0               (12, 7, 4, 1)
              1                (9, 2, 5, 2)
              2               (11, 4, 4, 3)
              3                (4, 1, 3, 0)
              4               (10, 5, 2, 3)
4             0                (8, 3, 3, 2)
              1                (8, 3, 2, 3)
              2               (12, 4, 7, 1)
     

In [528]:
# change order of columns 
def change_order():
    for l in league.values():
        modified = pd.read_csv('cleaned_data/{league}.csv'.format(league = l))[['tID', 'season', \
            'matches_played', 'wins', 'draws', 'losses', 'goals_scored', 'goals_conceded', 'goal_difference',\
                'league_position']]
        modified.to_csv('cleaned_data/{league}.csv'.format(league = l), index = False)
    for match in ['europa-league_match', 'champions-league_match']:
        match_df = pd.read_csv('cleaned_data/{c}.csv'.format(c = match))[['home_team', 'away_team',\
             'season', 'home_team_goal_count', 'away_team_goal_count']]
        match_df.to_csv('cleaned_data/{league}.csv'.format(league = match), index = False)  
    for team in ['europa-league_team', 'champions-league_team']:
        team_df = pd.read_csv('cleaned_data/{team}.csv'.format(team = team))[['tID', 'season', \
            'matches_played', 'wins', 'draws', 'losses', 'goals_scored', 'goals_conceded', 'goal_difference']]
        team_df.to_csv('cleaned_data/{team}.csv'.format(team = team), index = False)  

In [529]:
change_order()

In [530]:
pd.read_csv('cleaned_data/europa-league_team.csv')

,tID,season,matches_played,wins,draws,losses,goals_scored,goals_conceded,goal_difference
0,1,2017/2018,14,9,2,3,28,11,17
1,1,2018/2019,6,3,1,2,12,9,3
2,8,2017/2018,14,8,3,3,30,12,18
3,8,2018/2019,15,11,1,3,30,13,17
4,8,2019/2020,8,4,2,2,16,9,7
...,...,...,...,...,...,...,...,...,...
161,161,2016/2017,4,1,1,2,5,4,1
162,162,2013/2014,6,1,2,3,6,7,-1
163,163,2014/2015,12,6,3,3,24,18,6
164,163,2019/2020,10,5,2,3,15,13,2


In [531]:
pd.read_csv('cleaned_data/la-liga.csv')

,tID,season,matches_played,wins,draws,losses,goals_scored,goals_conceded,goal_difference,league_position
0,4,2010,38,6,12,20,36,70,-34,20
1,4,2013,38,11,7,20,43,71,-28,17
2,4,2014,38,8,8,22,35,64,-29,19
3,11,2010,38,18,4,16,59,55,4,6
4,11,2011,38,12,13,13,49,52,-3,10
...,...,...,...,...,...,...,...,...,...,...
195,157,2015,38,18,10,10,44,35,9,4
196,157,2016,38,19,10,9,56,33,23,5
197,157,2017,38,18,7,13,57,50,7,5
198,157,2018,38,10,14,14,49,52,-3,14
